# #3


# Fit and Predict Credit Information


I am going to fit and predit credit information of a competition, "Home Credit Default Risk" on Kaggle.


https://www.kaggle.com/c/home-credit-default-risk


## [Task 1] Create a Pipeline

In [1]:
import pandas as pd
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score
import numpy as np
import heapq
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/Users/kazukiegusa/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
# get the train and test dataset
train = pd.read_csv('"Home Credit Default Risk"_application_train.csv')
test = pd.read_csv('"Home Credit Default Risk"_application_test.csv')

# concatenate the train dataset with the test dataset
train_test = train.append(test)

# create dummies
new_train_test = pd.get_dummies(train_test)

#  compute the correlation coefficients
corr_mat = new_train_test.corr()

# split the concatenated dataset into the train dataset and the test dataset
train_input = new_train_test[0: train.shape[0]].copy()
test_input = new_train_test[train.shape[0]:].copy()

/Users/kazukiegusa/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
# create a pipeline by LightGBM
X = train_input.drop("TARGET",axis=1).values
y = train_input.loc[:,'TARGET'].values

lgbm_train = lgbm.Dataset(X,y)

lgbm_eval = lgbm.Dataset(X, y, reference=lgbm_train)

lgbm_params = {'objective':'binary',
               'boosting_type': 'rf',
               'subsample': 0.623,
               'colsample_bytree': 0.7,
               'num_leaves': 127,
               'max_depth': 8,
               'seed': 99,
               'bagging_freq': 1,
               'n_jobs': 4}

model = lgbm.train(lgbm_params, lgbm_train, valid_sets=lgbm_eval)

[1]	valid_0's binary_logloss: 0.262326
[2]	valid_0's binary_logloss: 0.25312
[3]	valid_0's binary_logloss: 0.251118
[4]	valid_0's binary_logloss: 0.250335
[5]	valid_0's binary_logloss: 0.250268
[6]	valid_0's binary_logloss: 0.250072
[7]	valid_0's binary_logloss: 0.249893
[8]	valid_0's binary_logloss: 0.249758
[9]	valid_0's binary_logloss: 0.249673
[10]	valid_0's binary_logloss: 0.249562
[11]	valid_0's binary_logloss: 0.249642
[12]	valid_0's binary_logloss: 0.24955
[13]	valid_0's binary_logloss: 0.249641
[14]	valid_0's binary_logloss: 0.249725
[15]	valid_0's binary_logloss: 0.249621
[16]	valid_0's binary_logloss: 0.249718
[17]	valid_0's binary_logloss: 0.249586
[18]	valid_0's binary_logloss: 0.24948
[19]	valid_0's binary_logloss: 0.249417
[20]	valid_0's binary_logloss: 0.249535
[21]	valid_0's binary_logloss: 0.249576
[22]	valid_0's binary_logloss: 0.249498
[23]	valid_0's binary_logloss: 0.249422
[24]	valid_0's binary_logloss: 0.24936
[25]	valid_0's binary_logloss: 0.24932
[26]	valid_0's

In [4]:
# compute feature importances

train_features = [f for f in train_input if f not in ['SK_ID_CURR','TARGET']]

fi_lgbm = pd.DataFrame()

fi_lgbm['importance_split'] = model.feature_importance(importance_type='split')
fi_lgbm['importance_gain'] = model.feature_importance(importance_type='gain')

feat_labels = train_input[train_features].columns[0:]

indices_s = np.argsort(fi_lgbm['importance_split'])[::-1]
indices_g = np.argsort(fi_lgbm['importance_gain'])[::-1]

for f in range(train_input[train_features].shape[1]):
    print('%2d) %-*s %f' % (f+1, 30, feat_labels[indices_s[f]], fi_lgbm['importance_split'][indices_s[f]]))
    
for f in range(train_input[train_features].shape[1]):
    print('%2d) %-*s %f' % (f+1, feat_labels[indices_g[f]], fi_lgbm['importance_gain'][indices_g[f]]))    

 1) ORGANIZATION_TYPE_Insurance    0.000000
 2) ORGANIZATION_TYPE_Construction 0.000000
 3) ORGANIZATION_TYPE_Electricity  0.000000
 4) ORGANIZATION_TYPE_Emergency    0.000000
 5) ORGANIZATION_TYPE_Government   0.000000
 6) ORGANIZATION_TYPE_Housing      0.000000
 7) ORGANIZATION_TYPE_Industry: type 1 0.000000
 8) ORGANIZATION_TYPE_Industry: type 10 0.000000
 9) ORGANIZATION_TYPE_Industry: type 11 0.000000
10) ORGANIZATION_TYPE_Industry: type 13 0.000000
11) ORGANIZATION_TYPE_Industry: type 2 0.000000
12) ORGANIZATION_TYPE_Industry: type 3 0.000000
13) ORGANIZATION_TYPE_Industry: type 5 0.000000
14) ORGANIZATION_TYPE_Industry: type 6 0.000000
15) ORGANIZATION_TYPE_Industry: type 7 0.000000
16) ORGANIZATION_TYPE_Industry: type 8 0.000000
17) ORGANIZATION_TYPE_Industry: type 9 0.000000
18) ORGANIZATION_TYPE_Kindergarten 0.000000
19) ORGANIZATION_TYPE_Medicine     0.000000
20) ORGANIZATION_TYPE_Military     0.000000
21) ORGANIZATION_TYPE_Mobile       0.000000
22) ORGANIZATION_TYPE_Other  

IndexError: index 244 is out of bounds for axis 0 with size 244

In [ ]:
# draw a graph of the feature importances

ranking = np.argsort(-fi_lgbm['importance_split'])

f, ax = plt.subplots(figsize=(90, 90)) 

colors = ['red' if (x>=min(heapq.nlargest(10, fi_lgbm['importance_split'][ranking]))) else 'grey' \
          for x in fi_lgbm['importance_split'][ranking]]

sns.barplot(x=fi_lgbm['importance_split'][ranking], y=train_input[train_features].columns.values[ranking-1], orient='h', palette=colors)

ax.set_xlabel('feature importance',fontsize=70)
ax.set_yticklabels(feat_labels[indices_s-1],fontsize=30)
plt.tight_layout()
plt.show()

## [Task 2] Create a Definition of the AUC

In [ ]:
# def of the AUC

## [Task 3] Preprocessing, Feature Enginnering and Fitting


### Focus on "EXT_SOURCE_1"

In [5]:
# check the contents of "EXT_SOURCE_1"

new_train_test.EXT_SOURCE_1

0        0.083037
1        0.311267
2             NaN
3             NaN
4             NaN
5             NaN
6        0.774761
7             NaN
8        0.587334
9             NaN
10       0.319760
11       0.722044
12       0.464831
13            NaN
14       0.721940
15       0.115634
16            NaN
17            NaN
18            NaN
19            NaN
20       0.565655
21       0.437709
22            NaN
23            NaN
24            NaN
25       0.561948
26            NaN
27            NaN
28       0.600396
29       0.297914
           ...   
48714    0.623896
48715    0.554915
48716    0.361450
48717    0.725281
48718         NaN
48719    0.498882
48720    0.587915
48721    0.204548
48722         NaN
48723    0.448657
48724    0.478887
48725    0.642719
48726    0.563175
48727    0.574515
48728    0.787271
48729         NaN
48730    0.440056
48731    0.573130
48732    0.527067
48733    0.363857
48734    0.410389
48735         NaN
48736    0.851722
48737    0.442558
48738    0

In [6]:
new_train_test.EXT_SOURCE_1.describe()

count    162345.000000
mean          0.501965
std           0.210045
min           0.013458
25%           0.335503
50%           0.506155
75%           0.673344
max           0.962693
Name: EXT_SOURCE_1, dtype: float64

In [7]:
# count the number of the missing values

new_train_test.EXT_SOURCE_1.isnull().sum()

193910

In [8]:
# get top 50 feature values strongly correlated with "EXT_SOURCE_1"

EXT_SOURCE_1_correlations = corr_mat.nlargest(51, "EXT_SOURCE_1")["EXT_SOURCE_1"].drop("EXT_SOURCE_1")

EXT_SOURCE_1_correlations_table = pd.DataFrame({
    'correlation': EXT_SOURCE_1_correlations
})

EXT_SOURCE_1_correlations_table

,correlation
CODE_GENDER_F,0.302426
NAME_INCOME_TYPE_Pensioner,0.285219
ORGANIZATION_TYPE_XNA,0.285086
DAYS_EMPLOYED,0.280783
EXT_SOURCE_2,0.212987
FLAG_DOCUMENT_6,0.184286
EXT_SOURCE_3,0.181390
AMT_GOODS_PRICE,0.177335
AMT_CREDIT,0.171269
NAME_HOUSING_TYPE_House / apartment,0.140927


    - There are no strong correlations.

In [9]:
# compare "EXT_SOURCE_1" with "TARGET"

new_train_test.EXT_SOURCE_1[new_train_test.TARGET==1].describe()

count    10054.000000
mean         0.386968
std          0.204729
min          0.014691
25%          0.221847
50%          0.361675
75%          0.537508
max          0.929394
Name: EXT_SOURCE_1, dtype: float64

In [10]:
new_train_test.EXT_SOURCE_1[new_train_test.TARGET==0].describe()

count    124079.000000
mean          0.511461
std           0.208804
min           0.014568
25%           0.345430
50%           0.517452
75%           0.682839
max           0.962693
Name: EXT_SOURCE_1, dtype: float64

    - I need to take the same number of samples from "EXT_SOURCE_1" whose "TARGET" values is 0 with the number of "EXT_SOURCE_1" whose "TARGET" values is 1.

In [11]:
# adjust the number of 2 kinds of samples

new_train_test.EXT_SOURCE_1[new_train_test.TARGET==0].sample(n=25000).describe()

count    10925.000000
mean         0.509279
std          0.208559
min          0.017756
25%          0.345495
50%          0.510651
75%          0.679302
max          0.943982
Name: EXT_SOURCE_1, dtype: float64

    - The mean of the samples whose "TARGET" value is 1 smaller than the samples whose "TARGET" value is 0.

In [12]:
# create a new feature value based on "EXT_SOURCE_1"

# add a spare column
new_train_test["ADD_EXT_SOURCE_1"] = float(0)

new_train_test.ADD_EXT_SOURCE_1

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
48714    0.0
48715    0.0
48716    0.0
48717    0.0
48718    0.0
48719    0.0
48720    0.0
48721    0.0
48722    0.0
48723    0.0
48724    0.0
48725    0.0
48726    0.0
48727    0.0
48728    0.0
48729    0.0
48730    0.0
48731    0.0
48732    0.0
48733    0.0
48734    0.0
48735    0.0
48736    0.0
48737    0.0
48738    0.0
48739    0.0
48740    0.0
48741    0.0
48742    0.0
48743    0.0
Name: ADD_EXT_SOURCE_1, Length: 356255, dtype: float64

In [13]:
# substitute 1 to the samples whose mean is less than the mean of the 2 kinds of samples whose "TARGET" values is 0 or 1.

new_train_test.loc[new_train_test.EXT_SOURCE_1<(new_train_test.EXT_SOURCE_1[new_train_test.TARGET==1].mean()+ \
                                                new_train_test.EXT_SOURCE_1[new_train_test.TARGET==0].sample(n=25000).mean())/2, 
                   "ADD_EXT_SOURCE_1"] = 1

new_train_test.ADD_EXT_SOURCE_1

0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       1.0
11       0.0
12       0.0
13       0.0
14       0.0
15       1.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       1.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       1.0
        ... 
48714    0.0
48715    0.0
48716    1.0
48717    0.0
48718    0.0
48719    0.0
48720    0.0
48721    1.0
48722    0.0
48723    1.0
48724    0.0
48725    0.0
48726    0.0
48727    0.0
48728    0.0
48729    0.0
48730    1.0
48731    0.0
48732    0.0
48733    1.0
48734    1.0
48735    0.0
48736    0.0
48737    1.0
48738    1.0
48739    0.0
48740    0.0
48741    0.0
48742    1.0
48743    0.0
Name: ADD_EXT_SOURCE_1, Length: 356255, dtype: float64

### Focus on "EXT_SOURCE_2"

In [14]:
# compare "EXT_SOURCE_2" with "TARGET"

new_train_test.EXT_SOURCE_2[new_train_test.TARGET==1].describe()

count    24773.000000
mean         0.410935
std          0.213107
min          0.000005
25%          0.238568
50%          0.440381
75%          0.594190
max          0.811870
Name: EXT_SOURCE_2, dtype: float64

In [15]:
new_train_test.EXT_SOURCE_2[new_train_test.TARGET==0].describe()

count    2.820780e+05
mean     5.234787e-01
std      1.862767e-01
min      8.173617e-08
25%      4.097866e-01
50%      5.739047e-01
75%      6.677051e-01
max      8.549997e-01
Name: EXT_SOURCE_2, dtype: float64

In [16]:
# adjust the number of 2 kinds of samples

new_train_test.EXT_SOURCE_2[new_train_test.TARGET==0].sample(25000).describe()

count    24952.000000
mean         0.523885
std          0.185434
min          0.000085
25%          0.411231
50%          0.573905
75%          0.667492
max          0.855000
Name: EXT_SOURCE_2, dtype: float64

In [17]:
# create a new feature value based on "EXT_SOURCE_2"

# add a spare column
new_train_test["ADD_EXT_SOURCE_2"] = float(0)

new_train_test.ADD_EXT_SOURCE_2

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
48714    0.0
48715    0.0
48716    0.0
48717    0.0
48718    0.0
48719    0.0
48720    0.0
48721    0.0
48722    0.0
48723    0.0
48724    0.0
48725    0.0
48726    0.0
48727    0.0
48728    0.0
48729    0.0
48730    0.0
48731    0.0
48732    0.0
48733    0.0
48734    0.0
48735    0.0
48736    0.0
48737    0.0
48738    0.0
48739    0.0
48740    0.0
48741    0.0
48742    0.0
48743    0.0
Name: ADD_EXT_SOURCE_2, Length: 356255, dtype: float64

In [18]:
# substitute 1 to the samples whose mean is less than the mean of the 2 kinds of samples whose "TARGET" values is 0 or 1.

new_train_test.loc[new_train_test.EXT_SOURCE_2<(new_train_test.EXT_SOURCE_2[new_train_test.TARGET==1].mean()+ \
                                                new_train_test.EXT_SOURCE_2[new_train_test.TARGET==0].sample(n=25000).mean())/2, 
                   "ADD_EXT_SOURCE_2"] = 1

new_train_test.ADD_EXT_SOURCE_2

0        1.0
1        0.0
2        0.0
3        0.0
4        1.0
5        1.0
6        0.0
7        0.0
8        1.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       1.0
16       1.0
17       0.0
18       0.0
19       0.0
20       1.0
21       1.0
22       1.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
48714    0.0
48715    0.0
48716    1.0
48717    0.0
48718    0.0
48719    1.0
48720    0.0
48721    0.0
48722    1.0
48723    0.0
48724    1.0
48725    1.0
48726    0.0
48727    1.0
48728    0.0
48729    0.0
48730    1.0
48731    1.0
48732    1.0
48733    1.0
48734    0.0
48735    1.0
48736    0.0
48737    0.0
48738    0.0
48739    0.0
48740    0.0
48741    0.0
48742    1.0
48743    1.0
Name: ADD_EXT_SOURCE_2, Length: 356255, dtype: float64

### Focus on "EXT_SOURCE_3"

In [19]:
# compare "EXT_SOURCE_3" with "TARGET"

new_train_test.EXT_SOURCE_3[new_train_test.TARGET==1].describe()

count    19148.000000
mean         0.390717
std          0.205810
min          0.000527
25%          0.223831
50%          0.379100
75%          0.553165
max          0.896010
Name: EXT_SOURCE_3, dtype: float64

In [20]:
new_train_test.EXT_SOURCE_3[new_train_test.TARGET==0].describe()

count    227398.000000
mean          0.520969
std           0.190465
min           0.000527
25%           0.385915
50%           0.546023
75%           0.673830
max           0.893976
Name: EXT_SOURCE_3, dtype: float64

In [21]:
# adjust the number of 2 kinds of samples

new_train_test.EXT_SOURCE_3[new_train_test.TARGET==0].sample(25000).describe()

count    20211.000000
mean         0.519444
std          0.191460
min          0.000527
25%          0.384207
50%          0.546023
75%          0.672243
max          0.882530
Name: EXT_SOURCE_3, dtype: float64

In [22]:
# create a new feature value based on "EXT_SOURCE_3"

# add a spare column
new_train_test["ADD_EXT_SOURCE_3"] = float(0)

new_train_test.ADD_EXT_SOURCE_3

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
48714    0.0
48715    0.0
48716    0.0
48717    0.0
48718    0.0
48719    0.0
48720    0.0
48721    0.0
48722    0.0
48723    0.0
48724    0.0
48725    0.0
48726    0.0
48727    0.0
48728    0.0
48729    0.0
48730    0.0
48731    0.0
48732    0.0
48733    0.0
48734    0.0
48735    0.0
48736    0.0
48737    0.0
48738    0.0
48739    0.0
48740    0.0
48741    0.0
48742    0.0
48743    0.0
Name: ADD_EXT_SOURCE_3, Length: 356255, dtype: float64

In [23]:
# substitute 1 to the samples whose mean is less than the mean of the 2 kinds of samples whose "TARGET" values is 0 or 1.

new_train_test.loc[new_train_test.EXT_SOURCE_3<(new_train_test.EXT_SOURCE_3[new_train_test.TARGET==1].mean()+ \
                                                new_train_test.EXT_SOURCE_3[new_train_test.TARGET==0].sample(n=25000).mean())/2, 
                   "ADD_EXT_SOURCE_3"] = 1

new_train_test.ADD_EXT_SOURCE_3

0        1.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       1.0
11       0.0
12       1.0
13       0.0
14       0.0
15       0.0
16       1.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       1.0
23       0.0
24       0.0
25       0.0
26       1.0
27       0.0
28       0.0
29       0.0
        ... 
48714    0.0
48715    0.0
48716    0.0
48717    0.0
48718    0.0
48719    0.0
48720    0.0
48721    1.0
48722    0.0
48723    1.0
48724    1.0
48725    0.0
48726    0.0
48727    0.0
48728    0.0
48729    0.0
48730    0.0
48731    0.0
48732    1.0
48733    0.0
48734    0.0
48735    0.0
48736    0.0
48737    1.0
48738    1.0
48739    0.0
48740    0.0
48741    1.0
48742    0.0
48743    1.0
Name: ADD_EXT_SOURCE_3, Length: 356255, dtype: float64

### Feature Engineering I Did Before

In [24]:
# handle the missing values of the following 7 columns by changing theirown missing values to 0s

edit_missing_values = {"OWN_CAR_AGE":0,"AMT_REQ_CREDIT_BUREAU_YEAR":0,"AMT_REQ_CREDIT_BUREAU_QRT":0,"AMT_REQ_CREDIT_BUREAU_MON":0,
                       "AMT_REQ_CREDIT_BUREAU_DAY":0,"AMT_REQ_CREDIT_BUREAU_HOUR":0,"AMT_REQ_CREDIT_BUREAU_WEEK":0}

edit_train_test = new_train_test.fillna(value=edit_missing_values)

edit_train_test

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ADD_EXT_SOURCE_1,ADD_EXT_SOURCE_2,ADD_EXT_SOURCE_3
0,24700.5,406597.5,351000.0,202500.000,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,1,1.0,1.0,1.0
1,35698.5,1293502.5,1129500.0,270000.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,1.0,0.0,0.0
2,6750.0,135000.0,135000.0,67500.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0.0,0.0,0.0
3,29686.5,312682.5,297000.0,135000.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0.0,0.0,0.0
4,21865.5,513000.0,513000.0,121500.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0.0,1.0,0.0
5,27517.5,490495.5,454500.0,99000.000,0.0,0.0,0.0,1.0,0.0,1.0,...,0,0,0,0,0,0,1,0.0,1.0,0.0
6,41301.0,1560726.0,1395000.0,171000.000,0.0,0.0,1.0,1.0,0.0,2.0,...,0,0,0,1,0,0,0,0.0,0.0,0.0
7,42075.0,1530000.0,1530000.0,360000.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0.0,0.0,0.0
8,33826.5,1019610.0,913500.0,112500.000,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,1,0.0,1.0,0.0
9,20250.0,405000.0,405000.0,135000.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0.0,0.0,0.0


In [25]:
# split the concatenated dataset into the train dataset and the test dataset

train_input2 = edit_train_test[0: train.shape[0]].copy()
test_input2 = edit_train_test[train.shape[0]:].copy()

In [26]:
# create a pipeline by LightGBM

X2 = train_input2.drop("TARGET",axis=1).values
y2 = train_input2.loc[:,'TARGET'].values

lgbm_train2 = lgbm.Dataset(X2,y2)

lgbm_eval2 = lgbm.Dataset(X2, y2, reference=lgbm_train2)

model2 = lgbm.train(lgbm_params, lgbm_train2, valid_sets=lgbm_eval2)

[1]	valid_0's binary_logloss: 0.262097
[2]	valid_0's binary_logloss: 0.25402
[3]	valid_0's binary_logloss: 0.252208
[4]	valid_0's binary_logloss: 0.252351
[5]	valid_0's binary_logloss: 0.251214
[6]	valid_0's binary_logloss: 0.250935
[7]	valid_0's binary_logloss: 0.250551
[8]	valid_0's binary_logloss: 0.250306
[9]	valid_0's binary_logloss: 0.24994
[10]	valid_0's binary_logloss: 0.249797
[11]	valid_0's binary_logloss: 0.249675
[12]	valid_0's binary_logloss: 0.249586
[13]	valid_0's binary_logloss: 0.249542
[14]	valid_0's binary_logloss: 0.249508
[15]	valid_0's binary_logloss: 0.249519
[16]	valid_0's binary_logloss: 0.249472
[17]	valid_0's binary_logloss: 0.249479
[18]	valid_0's binary_logloss: 0.249455
[19]	valid_0's binary_logloss: 0.24943
[20]	valid_0's binary_logloss: 0.249399
[21]	valid_0's binary_logloss: 0.249394
[22]	valid_0's binary_logloss: 0.249375
[23]	valid_0's binary_logloss: 0.249335
[24]	valid_0's binary_logloss: 0.249319
[25]	valid_0's binary_logloss: 0.249308
[26]	valid_0

In [27]:
# predict the test dataset
y_pred2 = model2.predict(X2)

# compute AUC
roc_auc_score(y2,y_pred2)

0.7511005315225298

In [28]:
# predict the test dataset
y_test_pred2 = model2.predict(test_input2.values, num_iteration=model2.best_iteration)

# submittion the result
sub = pd.read_csv('"Home Credit Default Risk"_sample_submission.csv')

sub['TARGET'] = y_test_pred2

sub.to_csv('"Home Credit Default Risk"_pipeline4.csv', index=False)